In [1]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import cv2


from torch.utils.data import DataLoader
from torch.nn.functional import binary_cross_entropy
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torch.nn import Linear, ReLU, CrossEntropyLoss, Conv2d, MaxPool2d, Module
import torch
import pandas as pd
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import math


In [13]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader
import cv2
import torchvision.transforms.functional as tx
from sklearn.utils import shuffle
import torch
class AL_Dataset():
    def __init__(self, mode):
        if(mode=="train"):
            self.AD_dir = np.array(os.listdir("D:/Project/data_crop/training_data/AD"))
            self.CN_dir = np.array(os.listdir("D:/Project/data_crop/training_data/CN"))
        else:
            self.AD_dir = np.array(os.listdir("D:/Project/data_crop/validation_data/AD"))
            self.CN_dir = np.array(os.listdir("D:/Project/data_crop/validation_data/CN"))
        self.AD = np.ones((len(self.AD_dir)))
        self.CN = np.zeros((len(self.CN_dir)))
        self.df = np.concatenate((self.AD_dir, self.CN_dir))
        self.label = np.concatenate((self.AD, self.CN))
        self.df, self.label = shuffle(self.df, self.label)
        self.mode = mode
        self.label_df = pd.read_csv("../data/B1_correction_new.csv")

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        if self.mode=="train":
            if self.label[idx]==1:
                image_path = "D:/Project/data_crop/training_data/AD/"+self.df[idx]

            else:
                image_path = "D:/Project/data_crop/training_data/CN/"+self.df[idx]

        else:
            if self.label[idx]==1:
                image_path = "D:/Project/data_crop/validation_data/AD/"+self.df[idx]

            else:
                image_path = "D:/Project/data_crop/validation_data/CN/"+self.df[idx]
#         print(image_path)
        patient_ID = image_path.split("/")[5].split(".")[0]
        index = list(self.label_df["Image Data ID"]).index(patient_ID)
        sex = self.label_df["Sex"][index]
        if sex =="M":
            label = 1
        else:
            label = 0
        # print(patient_ID)
        whole_image = nib.load(image_path)
        whole_image = np.array(whole_image.dataobj)
        image_arr = np.zeros((90, 256, 256, 3))
        for i in range((90)):            
            image = whole_image[i]
            image = cv2.resize(image, (256, 256), interpolation=cv2.INTER_AREA)
            image = normalize(image)
            image = image.reshape(256, 256, 1)
            image = np.concatenate((image, image, image), 2)
            image_arr[i] = image
        return image_arr, label
    
def normalize(arr):
    arr_min = np.min(arr)
    arr_max = np.max(arr)
    return (arr - arr_min) / (arr_max - arr_min)

In [3]:

def train(dataloader, model, loss_fn, optimizer, writer, epoch, level):

    model.train()
    total_loss = 0
    total_score = 0
    for batch, (X_arr, y) in enumerate(dataloader):
        X = X_arr[:,level]
        X= torch.moveaxis(X,3,1)
        X = X.float()
        y = y.long()
        X, y = X.to(device), y.to(device)

        X = X.requires_grad_()
        # Compute prediction error
        pred = model(X)
        total_score += (np.sum(y.cpu().detach().numpy()==np.argmax(pred.cpu().detach().numpy(), axis = 1))/y.shape[0])
        loss = loss_fn(pred, y)
        total_loss+=(abs(loss)/y.shape[0])
        loss.requires_grad_()
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 

        if batch % 25 == 0:
            loss, current = loss.item(), batch * len(X)
            print("total_loss : ", total_loss)
    writer.add_scalar("Loss/train", total_loss/len(dataloader), epoch)
    writer.add_scalar("Score/train", total_score/len(dataloader), epoch)
    return total_loss, total_score



In [4]:
def valid(dataloader, model, loss_fn, optimizer, writer, epoch, level):
    model.eval()

    size = len(dataloader.dataset)
    model.train()
    total_loss = 0
    total_score = 0
    for batch, (X_arr, y) in enumerate(dataloader):
        X = X_arr[:,level]
        X= torch.moveaxis(X,3,1)
        X = X.float()
        y = y.long()
        X, y = X.to(device), y.to(device)
        pred = model(X)
        total_score += (np.sum(y.cpu().detach().numpy()==np.argmax(pred.cpu().detach().numpy(), axis = 1))/y.shape[0])
        loss = loss_fn(pred, y)
        total_loss+=(abs(loss)/y.shape[0])

    writer.add_scalar("Loss/val", total_loss/len(dataloader), epoch)
    writer.add_scalar("Score/val", total_score/len(dataloader), epoch)
    return total_loss, total_score

In [18]:
train_dataset = AL_Dataset("train")
val_dataset = AL_Dataset("valid")
train_dataloader = DataLoader(train_dataset, batch_size = 1, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = 1, shuffle = True)
device = 'cuda' if torch.cuda.is_available() else "cpu"
weight_map = torch.zeros((2))
weight_map[0] = 1
weight_map[1] = 1
weight_map = weight_map.to(device)




In [19]:
image_arr, label = next(iter(train_dataloader))
print(device)

cuda


In [21]:
epochs = 20
# device = 'cuda' if torch.cuda.is_available() else "cpu"
date = "10_23_1"
writer = SummaryWriter(date)
print(device)
max_score=0
min_loss = 100
each_score = np.zeros((100))
if not os.path.exists("../data/models"):
    os.mkdir("../data/models")
for i in range(45, 90):
    print("training with ",i," level")
#     model = models.resnet101(pretrained=False)
#     model.fc = nn.Linear(2048, 2)
    model = models.resnet34(pretrained=False)
    model.fc = nn.Linear(512, 2)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
    loss_fn = nn.CrossEntropyLoss(weight=weight_map)
    min_loss = 100
    max_score = 0
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        total_loss_train, total_score_train = train(train_dataloader, model, loss_fn, optimizer, writer, t, i)
        print("training loss : ", total_loss_train/len(train_dataloader))
        print("training total score : ", total_score_train/len(train_dataloader))
        if(t%3==0): 
            with torch.no_grad():
                total_loss_val, total_score_val = valid(val_dataloader, model, loss_fn, optimizer, writer, t, i)
                print("validation loss : ", total_loss_val/len(val_dataloader))
                print("validation total score : ", total_score_val/len(val_dataloader))
                if(total_score_val/len(val_dataloader) > max_score ):
                    max_score = total_score_val/len(val_dataloader)
                    each_score[i] = max_score
                    print("#########save_best_model###########")
                    torch.save(model.state_dict(), "../data/models/level_"+str(i)+"_best"+".pth")
        torch.save(model.state_dict(), "../data/models/level_"+str(i)+"_last"+".pth")
print("Done!")

cuda
training with  45  level
Epoch 1
-------------------------------
total_loss :  tensor(0.7522, device='cuda:0', grad_fn=<AddBackward0>)
total_loss :  tensor(20.5144, device='cuda:0', grad_fn=<AddBackward0>)
total_loss :  tensor(38.2800, device='cuda:0', grad_fn=<AddBackward0>)
total_loss :  tensor(58.5595, device='cuda:0', grad_fn=<AddBackward0>)
total_loss :  tensor(78.2430, device='cuda:0', grad_fn=<AddBackward0>)
total_loss :  tensor(96.6650, device='cuda:0', grad_fn=<AddBackward0>)
total_loss :  tensor(115.6024, device='cuda:0', grad_fn=<AddBackward0>)
total_loss :  tensor(134.0063, device='cuda:0', grad_fn=<AddBackward0>)
total_loss :  tensor(154.7695, device='cuda:0', grad_fn=<AddBackward0>)
total_loss :  tensor(174.2768, device='cuda:0', grad_fn=<AddBackward0>)
total_loss :  tensor(190.2130, device='cuda:0', grad_fn=<AddBackward0>)
total_loss :  tensor(209.3362, device='cuda:0', grad_fn=<AddBackward0>)
total_loss :  tensor(227.2575, device='cuda:0', grad_fn=<AddBackward0>)
t

KeyboardInterrupt: 